In [1]:
!pip install ogb
!pip install dgl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 78 kB 5.7 MB/s 
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7048 sha256=65ca0b9ab0eba31fa3ee80074322cffa8c0891eb63b2250c834c622d85bda023
  Stored in directory: /root/.cache/pip/wheels/d6/64/cd/32819b511a488e4993f2fab909a95330289c3f4e0f6ef4676d
Successfully built littleutils
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.2 MB 19.8 MB/s 
     |████████████████████████████████| 281 kB 53.3 MB/s 
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8


In [4]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.12.0+cu113.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.12.0+cu113.html
!pip install torch-geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.12.0+cu113.html
     |████████████████████████████████| 7.9 MB 35.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.12.0+cu113.html
     |████████████████████████████████| 3.5 MB 33.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 407 kB 32.1 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.4-py3-none-any.whl size=616603 sha256=019ffffafa58f076f73e174aa4dd2c3762e24e539019b3c16045250720c113e4
  Stored in directory: /root/.cache/pip/wheels/18/a6/a4/ca18c3051fcead866fe7b85700ee2240d883562a1bc70ce421
Successfully built torch-geometric


In [6]:
import dgl
import torch
from dgl.dataloading import GraphDataLoader
from torch_geometric.data import DataLoader
from ogb.graphproppred import PygGraphPropPredDataset


dataset = PygGraphPropPredDataset(name = "ogbg-molhiv", root = 'dataset/')

split_idx = dataset.get_idx_split() 
train_loader = DataLoader(dataset[split_idx["train"]], batch_size=32, shuffle=True)
valid_loader = DataLoader(dataset[split_idx["valid"]], batch_size=32, shuffle=False)
test_loader = DataLoader(dataset[split_idx["test"]], batch_size=32, shuffle=False)

Downloaded 0.00 GB: 100%|██████████| 3/3 [00:02<00:00,  1.15it/s]
Processing...


Extracting dataset/hiv.zip
Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 41127/41127 [00:01<00:00, 28300.59it/s]


Converting graphs into PyG objects...


100%|██████████| 41127/41127 [00:02<00:00, 17196.05it/s]


Saving...


Done!
/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


'spilt_idx = dataset.get_idx_spilt()\n\ntrain_loader = GraphDataLoader(dataset[split_idx["train"]], batch_size=32, shuffle=True, collate_fn=collate_fn)\nvalid_loader = GraphDataLoader(dataset[split_idx["valid"]], batch_size=32, shuffle=False, collate_fn=collate_fn)\nteat_loader = GraphDataLoader(dataset[split_idx["teat"]], batch_size=32, shuffle=False, collate_fn=collate_fn)\n\ng,labels = next(iter(train_loader))\nprint(g)\nprint(labels)'

In [132]:
print(len(dataset))
print(dataset.num_classes)
print(dataset.num_node_features)
print(dataset[0])

41127
2
9
Data(edge_index=[2, 40], edge_attr=[40, 3], x=[19, 9], y=[1, 1], num_nodes=19)


In [121]:
import torch
import torch.nn.functional as F 
import torch_geometric.nn as pyg_nn
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
  def __init__(self):
      super().__init__()
      self.conv1 = GCNConv(dataset.num_node_features, 16)
      self.conv1_bn = pyg_nn.BatchNorm(16)
      self.conv2 = GCNConv(16, dataset.num_classes)
    
  def forward(self, data):
    x, edge_index, batch = data.x, data.edge_index, data.batch
    x = x.type(torch.float32)

    x = self.conv1(x, edge_index)
    x = F.relu(self.conv1_bn(x))
    x = F.dropout(x, training=self.training)
    x = self.conv2(x, edge_index)
    x = pyg_nn.global_max_pool(x, batch)

    return F.log_softmax(x, dim=1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(100):
  loss_all = 0
  for data in train_loader:
    optimizer.zero_grad()
    out = model(data)
    label = data.y
    label = label.squeeze(1)
    loss = F.nll_loss(out, label)
    loss.backward()
    optimizer.step()
  if epoch % 20 == 0:
    print(loss.item())


0.05466970056295395
0.03994530439376831
0.6641016006469727
0.053426384925842285
0.040232617408037186


In [151]:
from sklearn.metrics import accuracy_score

model.eval()

correct = 0

for data in test_loader:
    output = model(data)
    pred = output.max(dim=1)[1]
    data_y = data.y.reshape(-1)
    correct += (pred == data_y).sum().item()

print('準確率: {:07f}'.format(correct / len(test_loader.dataset)))

準確率: 0.968393
